<a href="https://colab.research.google.com/github/pkams/coursera_TensorFlowSpecialization/blob/master/Modulo3_4_TextGenerator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install wget

  Created wheel for wget: filename=wget-3.2-cp36-none-any.whl size=9682 sha256=93020f095cd80bf3fe43695de436d7169c8ab77c9a1a0cfc132724981153137a
  Stored in directory: /root/.cache/pip/wheels/40/15/30/7d8f7cea2902b4db79e3fea550d7d7b85ecb27ef992b618f3f
Successfully built wget


In [13]:
import tensorflow as tf
import pandas as pd
import numpy as np
import tensorflow.keras.utils as ku 

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [3]:
!wget --no-check-certificate \
    https://storage.googleapis.com/laurencemoroney-blog.appspot.com/sonnets.txt \
    -O /tmp/sonnets.txt

--2020-06-25 20:58:19--  https://storage.googleapis.com/laurencemoroney-blog.appspot.com/sonnets.txt
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.68.128, 172.217.194.128, 74.125.200.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.68.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 93578 (91K) [text/plain]
Saving to: ‘/tmp/sonnets.txt’

/tmp/sonnets.txt    100%[===================>]  91.38K  --.-KB/s    in 0.001s  

2020-06-25 20:58:19 (111 MB/s) - ‘/tmp/sonnets.txt’ saved [93578/93578]



In [4]:
data = open('/tmp/sonnets.txt').read()

corpus = data.lower().split('\n')

np.shape(corpus)

(2159,)

In [5]:
# Tokenizador
tokenizer = Tokenizer()
tokenizer.fit_on_texts(corpus)
word_index = tokenizer.word_index
total_words = len(word_index) +1
print('Total de palavras: ', total_words)

Total de palavras:  3211


In [46]:
# Criando sequencias de inputs usando os tokens
input_sequences = []
for line in corpus:
  # Tokeniza linha a linha e salva na token_list
  # o [0] é pra tirar o valor de dentro da lista e salvá-lo
  token_list = tokenizer.texts_to_sequences([line])[0]
  # Irá iterar sobre uma lista de 1 até tamanho da token_list
  for i in range(1, len(token_list)):
    n_gram_sequence = token_list[:i+1]
    input_sequences.append(n_gram_sequence)

# Pad Sequences
# Achando maior comprimento dentro da lista
max_sequences_len = max([len(x) for x in input_sequences])
# Fazendo padding
input_sequences = pad_sequences(input_sequences, maxlen=max_sequences_len, padding='pre')

# Criando X e y
X, y = input_sequences[:,:-1], input_sequences[:, -1]
y = ku.to_categorical(y, num_classes=total_words)

In [56]:
# Criando modelo
model = tf.keras.models.Sequential([
        # Dimensão de entrada: total de palavras
        # Dimensão de saída: 100
        # Tamanho da entrada: X                            
        tf.keras.layers.Embedding(total_words, 100, input_length=X.shape[1]),
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(150, return_sequences=True)),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.LSTM(100),
        tf.keras.layers.Dense(total_words/2, activation='relu', kernel_regularizer=tf.keras.regularizers.l1(0.01)),
        tf.keras.layers.Dense(total_words, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [61]:
history = model.fit(predictors, label, epochs=100, verbose=1)

Epoch 1/100
484/484 [==============================] - 6s 12ms/step - loss: 3.5941 - accuracy: 0.3201
Epoch 2/100
484/484 [==============================] - 6s 12ms/step - loss: 3.5747 - accuracy: 0.3233
Epoch 3/100
484/484 [==============================] - 6s 12ms/step - loss: 3.5721 - accuracy: 0.3224
Epoch 4/100
484/484 [==============================] - 6s 12ms/step - loss: 3.5561 - accuracy: 0.3315
Epoch 5/100
484/484 [==============================] - 6s 12ms/step - loss: 3.5446 - accuracy: 0.3308
Epoch 6/100
484/484 [==============================] - 6s 12ms/step - loss: 3.5335 - accuracy: 0.3318
Epoch 7/100
484/484 [==============================] - 6s 12ms/step - loss: 3.5128 - accuracy: 0.3365
Epoch 8/100
484/484 [==============================] - 6s 12ms/step - loss: 3.4965 - accuracy: 0.3451
Epoch 9/100
484/484 [==============================] - 6s 12ms/step - loss: 3.4767 - accuracy: 0.3459
Epoch 10/100
484/484 [==============================] - 6s 12ms/step - loss: 3.459

In [62]:
# Testando predição de novo texto

seed_text = "Help me Obi Wan Kenobi, you're my only hope"
next_words = 100
  
for _ in range(next_words):
	token_list = tokenizer.texts_to_sequences([seed_text])[0]
	token_list = pad_sequences([token_list], maxlen=max_sequences_len-1, padding='pre')
	predicted = np.argmax(model.predict(token_list, verbose=0), axis = -1)
	output_word = ""
	for word, index in tokenizer.word_index.items():
		if index == predicted:
			output_word = word
			break
	seed_text += " " + output_word
print(seed_text)

Help me Obi Wan Kenobi, you're my only hope receives light confined memory shalt then thee is i then i in thee forbear to thou joy more ' i in men let joy you control i store her beauty most shalt i say born day that been is in for my liberty virtue hell is action warm less thought the ornament itself more virtue shall thought buried art ever can evermore disabled physic the time day be in at such woe did heinous fell more brow ornament your warm to thou warm in time the style had lawful best matcheth let mine than thee shall know doth reason no
